### 네이버 뉴스 섹션별 headline 원문 기사 크롤링
- 기사제목, 상세보기, url, 신문사, 작성 날짜 등

In [1]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# 수집할 기사의 url 
head_L_df = pd.read_csv('./crawl_data/naver_news_headline.csv', index_col=0)
head_L_df.tail()

,head_line,news_link,section
47,"우라늄 싹쓸이하는 중국 기업들…""서방에 혼란올 것"" 경고",https://n.news.naver.com/mnews/article/015/000...,세계
48,"WHO 총장 ""이·팔 휴전 결의안 안보리 부결 깊은 유감""",https://n.news.naver.com/mnews/article/052/000...,세계
49,필리핀 선박에 ‘물대포’ 퍼부은 중국 해경…이틀째 책임 공방 이어져 [현장영상],https://n.news.naver.com/mnews/article/056/001...,세계
50,"송혜교·엄정화에 놀란 BBC…“30살 되면 여주인공 못했던 韓, 달라졌다”",https://n.news.naver.com/mnews/article/081/000...,세계
51,中 디플레 위기…물가 또 마이너스,https://n.news.naver.com/mnews/article/015/000...,세계


### 예시 코드 작성

In [5]:
url = head_L_df['news_link'][50]
url

'https://n.news.naver.com/mnews/article/081/0003415538?sid=104'

In [6]:
# 뉴스에 대한 소스 요청
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
res = requests.get(url, headers=headers)

In [9]:
html = res.text
bs_obj = BeautifulSoup(html, 'html.parser')

In [10]:
bs_obj

<!DOCTYPE html>

<html data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport">
<meta content="송혜교·엄정화에 놀란 BBC…“30살 되면 여주인공 못했던 韓, 달라졌다”" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="https://n.news.naver.com/mnews/article/081/0003415538?sid=104" property="og:url"/>
<meta content="https://imgnews.pstatic.net/image/081/2023/12/11/0003415538_001_20231211113501133.jpg?type=w800" property="og:image"/>
<meta content="한국 드라마 속 여성 주인공이 사랑에 기대는 신데렐라 캐릭터에서 벗어나 복수·성공·초능력 등의 강렬한 서사를 가진 독창적인 인물로 변모하고 있다는 외신 분석이 나왔다. 지난 10일(현지시간) 영국 BBC 방송은 관련" property="og:description"/>
<meta content="서울신문 | 네이버" property="og:article:author"/>
<meta content="summary_large_image" name="tw

In [15]:
# 추출하려고 하는 뉴스 기사 영역 한정
content = bs_obj.find("div",{'id':'ct'})
# 기사 작성 신문사
news_com = content.find('a').img['alt']
# 기사 작성자 및 입력날짜


In [22]:
info = content.select('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans')
info

[<div class="media_end_head_info nv_notrans">
 <div class="media_end_head_info_datestamp">
 <div class="media_end_head_info_datestamp_bunch">
 <em class="media_end_head_info_datestamp_term">입력</em><span class="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME" data-date-time="2023-12-11 11:14:01">2023.12.11. 오전 11:14</span>
 </div>
 <div class="media_end_head_info_datestamp_bunch">
 <em class="media_end_head_info_datestamp_term">수정</em><span class="media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME" data-modify-date-time="2023-12-11 11:35:01">2023.12.11. 오전 11:35</span>
 </div>
 <a class="media_end_head_origin_link" data-clk="are.ori" data-extra='{"lk":{"oid":"081","aid":"0548ada0e31392a0"}}' data-gdid="081" href="https://www.seoul.co.kr/news/newsView.php?id=20231211500072&amp;wlog_tag3=naver" target="_blank">기사원문</a>
 </div> <div class="media_end_head_journalist">
 <a class="media_end_head_journalist_box _LAZY_LOADING_WRAP" data-clk="ummore" href="https://media.naver.com

In [53]:
# 기사 입력 및 수정시간/작성기자/좋아요 수/댓글 수
css = '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp'
#AtributeError: ResultSet object has no attribute 'select' : select() 모듈 사용 불가
#info는 resultset 객체 임(원 소스에서 추출된 data) : select 사용 불가(list 형태)
info1 = info[0].select(css)
info2 = info[0].select('.media_end_head_info_datestamp')
date_all = info2[0].findAll('div',{'class':'media_end_head_info_datestamp_bunch'})
for date in date_all  :
    print(date.text)
ori_date = date_all[0].text.replace('\n','')[2:] 
corr_date = date_all[1].text.replace('\n','')[2:]


입력2023.12.11. 오전 11:14


수정2023.12.11. 오전 11:35



In [54]:
# 기자이름
# #ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_journalist > a > em
name=info[0].find('div',{'class':'media_end_head_journalist'}).em.text

In [40]:
info[0]

<div class="media_end_head_info nv_notrans">
<div class="media_end_head_info_datestamp">
<div class="media_end_head_info_datestamp_bunch">
<em class="media_end_head_info_datestamp_term">입력</em><span class="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME" data-date-time="2023-12-11 11:14:01">2023.12.11. 오전 11:14</span>
</div>
<div class="media_end_head_info_datestamp_bunch">
<em class="media_end_head_info_datestamp_term">수정</em><span class="media_end_head_info_datestamp_time _ARTICLE_MODIFY_DATE_TIME" data-modify-date-time="2023-12-11 11:35:01">2023.12.11. 오전 11:35</span>
</div>
<a class="media_end_head_origin_link" data-clk="are.ori" data-extra='{"lk":{"oid":"081","aid":"0548ada0e31392a0"}}' data-gdid="081" href="https://www.seoul.co.kr/news/newsView.php?id=20231211500072&amp;wlog_tag3=naver" target="_blank">기사원문</a>
</div> <div class="media_end_head_journalist">
<a class="media_end_head_journalist_box _LAZY_LOADING_WRAP" data-clk="ummore" href="https://media.naver.com/journalist

In [41]:
# js에 의해서 브라우저가 표현하는 내용이기 때문에 프로그램 코드로 요청한 소스에는 포함되어 있지 않음
# 좋아요 / 댓글 수
# 좋아요 / 댓글 수는 동적크롤링을 진행해야 얻어올 수 있음
css='#commentFontGroup > div.media_end_head_info_variety_likeit._LIKE_HIDE.as_likeit_improve > div > a > span.u_likeit_text'
info[0].select(css)

[<span class="u_likeit_text _count">추천</span>]

In [52]:
# 기사 제목 추출
content.select('#title_area') # 리스트 반환
title = content.find('h2',{'id':'title_area'}).text # 텍스트로 반환

In [50]:
# 기사 내용 추출
cnt=content.select('#dic_area')[0].text.replace('\n','').replace('\t','')

In [55]:
title, name, ori_date, corr_date, cnt

('송혜교·엄정화에 놀란 BBC…“30살 되면 여주인공 못했던 韓, 달라졌다”',
 '김민지 기자',
 '2023.12.11. 오전 11:14',
 '2023.12.11. 오전 11:35',
 '넷플릭스 ‘더 글로리’, JTBC ‘닥터 차정숙’한국 드라마 속 여성 주인공이 사랑에 기대는 신데렐라 캐릭터에서 벗어나 복수·성공·초능력 등의 강렬한 서사를 가진 독창적인 인물로 변모하고 있다는 외신 분석이 나왔다.지난 10일(현지시간) 영국 BBC 방송은 관련 보도에서 “현재 많은 K드라마(한국 드라마)에는 사회와 미디어 관행의 중대한 변화를 반영하는 복잡하고 강력한 여성 캐릭터가 등장한다”고 보도했다.BBC는 “K드라마에서 여성의 역할이 항상 흥미로운 것은 아니었다”면서 지난 2009년 방영된 KBS 드라마 ‘꽃보다 남자’의 금잔디 캐릭터를 예로 들었다. 꽃보다남자는 버릇없는 재벌 상속자와 용감한 서민 소녀가 사랑에 빠지게 되는 내용을 담은 일본 만화 원작 드라마로 당시 34.8%(TNS미디어코리아, 전국 기준)의 높은 시청률을 기록하며 큰 사랑을 받았다.매체는 K드라마 속 변화된 여성 캐릭터의 대표적인 예로 넷플릭스 드라마 ‘더 글로리’와 ENA 드라마 ‘이상한 변호사 우영우’를 꼽았다.BBC는 “올해 최고의 히트작 중 하나인 ‘더 글로리’는 괴롭힘에 맞서 복수하는 여성의 이야기를 담았고, 역시 큰 인기를 끌었던 ‘이상한 변호사 우영우’에는 자폐증이 있는 여성 변호사가 등장했다”고 소개했다.또 매체는 넷플릭스 드라마 ‘마이네임’을 언급하며 “요즘 K드라마에서는 폭력을 행사하는 여성도 나온다”고 밝혔다. ‘마이네임’은 아버지를 죽인 범인을 찾기 위해 경찰에 잠입한 딸의 복수를 다룬 드라마로, 여주인공의 강도 높은 액션과 정사 장면도 등장한다.지난 6월 인기리에 종영한 JTBC 드라마 ‘닥터 차정숙’도 언급됐다. 20년 넘게 가족만을 위해 살아온 가정주부가 의사로 새로운 삶을 살게 되는 과정을 그린 ‘닥터 차정숙’의 주연 엄정화 배우는 BBC와의 인터뷰에서 한국 드라

### 모든 헤드라인 기사 저장
- 빈 df 생성

In [57]:
title, name, ori_date, corr_date, cnt, news_com
news_content = pd.DataFrame({'title':[], 'writer':[], 'ori_date':[], 'corr_date':[], 'content':[], 'jornul':[]})
news_content

,title,writer,ori_date,corr_date,content,jornul


In [58]:
title, name, ori_date, corr_date, cnt, news_com

('송혜교·엄정화에 놀란 BBC…“30살 되면 여주인공 못했던 韓, 달라졌다”',
 '김민지 기자',
 '2023.12.11. 오전 11:14',
 '2023.12.11. 오전 11:35',
 '넷플릭스 ‘더 글로리’, JTBC ‘닥터 차정숙’한국 드라마 속 여성 주인공이 사랑에 기대는 신데렐라 캐릭터에서 벗어나 복수·성공·초능력 등의 강렬한 서사를 가진 독창적인 인물로 변모하고 있다는 외신 분석이 나왔다.지난 10일(현지시간) 영국 BBC 방송은 관련 보도에서 “현재 많은 K드라마(한국 드라마)에는 사회와 미디어 관행의 중대한 변화를 반영하는 복잡하고 강력한 여성 캐릭터가 등장한다”고 보도했다.BBC는 “K드라마에서 여성의 역할이 항상 흥미로운 것은 아니었다”면서 지난 2009년 방영된 KBS 드라마 ‘꽃보다 남자’의 금잔디 캐릭터를 예로 들었다. 꽃보다남자는 버릇없는 재벌 상속자와 용감한 서민 소녀가 사랑에 빠지게 되는 내용을 담은 일본 만화 원작 드라마로 당시 34.8%(TNS미디어코리아, 전국 기준)의 높은 시청률을 기록하며 큰 사랑을 받았다.매체는 K드라마 속 변화된 여성 캐릭터의 대표적인 예로 넷플릭스 드라마 ‘더 글로리’와 ENA 드라마 ‘이상한 변호사 우영우’를 꼽았다.BBC는 “올해 최고의 히트작 중 하나인 ‘더 글로리’는 괴롭힘에 맞서 복수하는 여성의 이야기를 담았고, 역시 큰 인기를 끌었던 ‘이상한 변호사 우영우’에는 자폐증이 있는 여성 변호사가 등장했다”고 소개했다.또 매체는 넷플릭스 드라마 ‘마이네임’을 언급하며 “요즘 K드라마에서는 폭력을 행사하는 여성도 나온다”고 밝혔다. ‘마이네임’은 아버지를 죽인 범인을 찾기 위해 경찰에 잠입한 딸의 복수를 다룬 드라마로, 여주인공의 강도 높은 액션과 정사 장면도 등장한다.지난 6월 인기리에 종영한 JTBC 드라마 ‘닥터 차정숙’도 언급됐다. 20년 넘게 가족만을 위해 살아온 가정주부가 의사로 새로운 삶을 살게 되는 과정을 그린 ‘닥터 차정숙’의 주연 엄정화 배우는 BBC와의 인터뷰에서 한국 드라

### 함수 구성
- 함수명 : content_crawl(url)
- 반환 : title, name, ori_date, corr_date, cnt, news_com
- 호출 : 변수 = content_crawl(기사url)
    - 빈 df에 행으로 삽입 

### 개별 실습
- 위의 코드 활용해서 함수 구성

In [ ]:
def content_crawl(url) :
    # 소스 가져오기, header 포함

    # 파서기(bs4 객체) 생성

    # 제목 추출
    # 기자 이름 추출
    # 날짜 추출
    # 내용 추출
    # 언론사 추출

    return title, name, ori_date, corr_date, cnt, news_com

In [ ]:
# 아래 코드로 함수 테스트 , 슬랙으로 제출
url = head_L_df['news_link'][50]
content_crawl(url)